# Рекомендация тарифов

__Содержание проекта__

[Шаг 1: открываем данные и изучаем общую информацию](#step1)
- [Делаем промежуточный вывод](#step1_last)

[Шаг 2: разбиваем данные на выборки](#step2)
- [Описываем подход](#step2_1)
- [Разбиваем на обучающую, валидационную и тестовую выборки](#step2_2)
- [Делаем dummy выборку для последующей проверки на адекватность](#step2_3)
- [Делаем промежуточный вывод](#step2_last)

[Шаг 3: исследуем модели классификации](#step3)
- [Модель решающее дерево](#step3_1)
- [Модель случайный лес](#step3_2)
- [Модель логистическая "регрессия"](#step3_3)
- [Делаем промежуточный вывод](#step3_last)

[Шаг 4: проверьте модель на тестовой выборке](#step4)
- [Делаем промежуточный вывод](#step4_last)

[Шаг 5: проверьте модели на адекватность](#step5)
- [Делаем промежуточный вывод](#step5_last)

[Шаг 6: делаем общий вывод](#step6)

<a id='step1'></a>

## 1. Откройте и изучите файл

_Импортируем библиотеки, которые нам понядобятся для работы по этому проекту._

In [1]:
import pandas as pd
from IPython.display import display
import random
import warnings
import numpy as np

#библиотеки моделей машинного обучения
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.dummy import DummyClassifier
from joblib import dump
import joblib

In [2]:
data = pd.read_csv("/datasets/users_behavior.csv")

In [3]:
display(data)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


<a id='step1_last'></a>

#### Вывод:
Мы открыли и изучили файл - предобработка данных не требуется. С файлом можно работать дальше, чтобы решить основную задачу построения рекомендательной модели тарифа.
___
В файле представлены следующие `признаки`. 
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
___

Целевой признак - `is_ultra`, его обозначим как `target`.
Все остальные признаки обозначим как `features`.
___
Нам необходимо разбить модель по соотношению 3/1/1 на 3 выборки: обучающую, валидационну и тестовую.
__Займемся этим в следующем разделе.__

<a id='step2'></a>

## 2. Разбейте данные на выборки

<a id='step2_1'></a>

_Текущая задача машинного обучения из разряда __обучение с учителем__ поскольку нам известен целевой признак. И это __задача классификации__, поскольку целевой признак из двух типов `ultra` или `smart`. А так как спрятанной тестовой выборки нет, то мы разбиваем исходные данные в соотношении __3/1/1__._

In [5]:
print("Общая выборка состоит из {} объектов.". format(round(len(data))))
print()
print("Значит, наша обучающая выборка должна быть 60%: {} объектов".format(round(len(data)*0.6)))
print("Валидационная выборка должна быть 20%: {} объектов".format(round(len(data)*0.2)))
print("Тестовая выборка должна быть 20%: {} объектов".format(round(len(data)*0.2)))

Общая выборка состоит из 3214 объектов.

Значит, наша обучающая выборка должна быть 60%: 1928 объектов
Валидационная выборка должна быть 20%: 643 объектов
Тестовая выборка должна быть 20%: 643 объектов


<a id='step2_2'></a>

_Сначала разобьем выборку используя метод из библиотеки `sklearn`, чтобы выделить валидационную выборку и совместно тестовую и обучающую. Потом повторно используем этот же метод для финального разделения на тестовую и обучающую выборки._ 

In [6]:
data_train_plus_test, data_valid = train_test_split(data, test_size=0.2, random_state=12345)
data_train, data_test = train_test_split(data_train_plus_test, test_size=0.25, random_state=12345)

_Проверим получились ли у нас запланированные длины выборок._

In [7]:
print("Обучающая выборка: {} объектов".format(len(data_train)))
print("Валидационная выборка: {} объектов".format(len(data_valid)))
print("Тестовая выборка: {} объектов".format(len(data_test)))

Обучающая выборка: 1928 объектов
Валидационная выборка: 643 объектов
Тестовая выборка: 643 объектов


_Отделим целевые признаки в каждой из выборок._

In [8]:
target_train = data_train['is_ultra']
features_train = data_train.drop('is_ultra', axis = 1)
target_valid = data_valid['is_ultra']
features_valid = data_valid.drop('is_ultra', axis = 1)
target_test = data_test['is_ultra']
features_test = data_test.drop('is_ultra', axis = 1)

<a id='step2_3'></a>

_Составим также `dummy` выборку для последующего сравнения на адекватность. Используем модуль `random` для случайного проставления признака тарифа. Эта модель будет равносильна подкидыванию монетки._

In [9]:
#coin_toss
target_dummy = []

for i in range(len(target_test)):
    target_dummy.append(random.randint(0,1))

<a id='step2_last'></a>

#### Вывод:
1. Мы разбили данные на 3 выборки согласно правилу `3/1/1` поскольку у нас нет спрятанной тестовой выборки.
1. Мы также добавили `dummy` выборку с целевым признаком (0,1) по тарифу ультра.
___
Далее исследуем 3 модели классификации:
- решающее дерево;
- случайный лес;
- логистическая регрессия.

<a id='step3'></a>

## 3. Исследуйте модели

_Исследуем 3 вида моделей:_
- решающее дерево, 
- случайный лес, 
- логистическую регрессию.
___
_Оценим показатели качества (`accuracy`, `precision`)_

<a id='step3_1'></a>

### 3.1 Модель _решающее дерево_

_Оцениваем влияние на качество (`accuracy`) разную глубину дерева от 1 до 10._

In [10]:
#модель и прогноз
accuracy_tree_0 = 0

for depth in range(1,11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model_tree.fit(features_train, target_train)
    prediction_tree = model_tree.predict(features_valid)
    accuracy_tree_1 = accuracy_score(target_valid, prediction_tree)
    if accuracy_tree_1 > accuracy_tree_0:
        best_accuracy_tree = accuracy_tree_1
        accuracy_tree_0 = accuracy_tree_1
        best_depth = depth
        joblib.dump(model_tree, 'model_tree.joblib')
print("Наибольшая точность 'решающего дерева' accuracy: {} достигается при depth: {} ".format(round(best_accuracy_tree,5), best_depth))

Наибольшая точность 'решающего дерева' accuracy: 0.78849 достигается при depth: 5 


_Оценим показатель качества `precision` для модели решающего дерева с 5ю уровнями и полученной максимальной оценкой `accuracy`._

In [11]:
precision_tree = precision_score(target_valid, prediction_tree, average='binary')
print("Доля объектов отмеченных моделью 'решаюшее дерево' как ультра действительно является ультра:", round(precision_tree,5))

Доля объектов отмеченных моделью 'решаюшее дерево' как ультра действительно является ультра: 0.67883


<a id='step3_2'></a>

### 3.2 Модель _случайный лес_

_Оцениваем влияние на качество (`accuracy`) лес с количеством деревьев от 10 до 50 (с шагом 10) и глубиной от 1 до 5._

In [12]:
#модель и прогноз
accuracy_forest_0 = 0

for est in range(10, 51, 10):
    for depth in range(1,6):
        model_forest = RandomForestClassifier(random_state=123456, n_estimators = est, max_depth = depth)
        model_forest.fit(features_train, target_train)
        prediction_forest = model_forest.predict(features_valid)
        accuracy_forest_1 = accuracy_score(target_valid, prediction_forest)
        if accuracy_forest_1 > accuracy_forest_0:
            best_accuracy_forest = accuracy_forest_1
            accuracy_forest_0 = accuracy_forest_1
            best_depth_forest = depth
            best_est = est
            joblib.dump(model_forest, 'model_forest.joblib')
print("Наибольшая точность 'решающего дерева' accuracy: {} достигается при depth: {} и количестве эстиматоров: {}.".format(round(best_accuracy_forest,5), best_depth_forest, best_est))

Наибольшая точность 'решающего дерева' accuracy: 0.79471 достигается при depth: 5 и количестве эстиматоров: 30.


_Оценим показатель качества `precision` для модели случайного леса с 5ю уровнями и 30 эстиматорами и полученной максимальной оценкой `accuracy`._

In [13]:
precision_forest = precision_score(target_valid, prediction_forest, average='binary')
print("Доля объектов отмеченных моделью 'случайный лес' как ультра действительно является ультра:", round(precision_forest,5))

Доля объектов отмеченных моделью 'случайный лес' как ультра действительно является ультра: 0.76033


<a id='step3_3'></a>

### 3.3 Модель _логистическая "регрессия"_

_Оцениваем влияние на качество (`accuracy`)_

In [14]:
#модель и прогноз
warnings.filterwarnings('ignore')
model_logistic = LogisticRegression(random_state=1234567)
model_logistic.fit(features_train, target_train)
prediction_logistic = model_logistic.predict(features_valid)
accuracy_logistic = accuracy_score(target_valid, prediction_logistic)
joblib.dump(model_logistic, 'model_logistic.joblib')
print("Точность 'логистической модели' accuracy: {} ".format(round(accuracy_logistic,5)))

Точность 'логистической модели' accuracy: 0.70295 


_Оценим показатель качества `precision` для логистической модели с полученной оценкой `accuracy`._

In [15]:
precision_logistic = precision_score(target_valid, prediction_logistic, average='binary')
print("Доля объектов отмеченных моделью 'случайный лес' как ультра действительно является ультра:", round(precision_logistic,5))

Доля объектов отмеченных моделью 'случайный лес' как ультра действительно является ультра: 0.69231


_Сделаем небольшой dataframe для легкого сравнения моделей_

In [16]:
model_comparision = pd.DataFrame({"model": ["Решающее дерево", "Случайный лес", "Логистическая"], 
                                  "accuracy_valid": [best_accuracy_tree, best_accuracy_forest, accuracy_logistic],
                                  "precision_valid": [precision_tree, precision_forest, precision_logistic]
                                 })

In [17]:
print("_ _ _ _ _ Табл.№1.Сравнение моделей (до)_ _ _ _ _ _ ")
print("_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ")
print(model_comparision)

_ _ _ _ _ Табл.№1.Сравнение моделей (до)_ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
             model  accuracy_valid  precision_valid
0  Решающее дерево        0.788491         0.678832
1    Случайный лес        0.794712         0.760331
2    Логистическая        0.702955         0.692308


<a id='step3_last'></a>

#### Вывод:
- Для каждой из 3х видов моделей мы определили наилучшие `гиперпараметры` перебором разных значений в цикле.
___
Для "Решающего дерева" - максимальное accuracy достигается при `depth` равным 5.
Для "Случайного леса" - при `depth` также равным 5 и количестве деревьев (`estimators`) 30.
Для "Логистической регрессии" - гиперпараметры не задаются.
___
- Сравнение в Таблице №1 выше показывает, что по показателям `accuracy` и `precision` наилучшей из 3х моделей в данном проекте является `Случайный лес`. Дальше продолжим работу с этой моделью.

<a id='step4'></a>

## 4. Проверьте модель на тестовой выборке

_Для проверки модели `Случайный лес` на тестовой выборке оценим показатель `accuracy` и `precision`._

In [18]:
model_forest = joblib.load('model_forest.joblib')
prediction_forest_test = model_forest.predict(features_test)
accuracy_forest_test = accuracy_score(target_test, prediction_forest_test)
precision_forest_test = precision_score(target_test, prediction_forest_test, average='binary')

In [19]:
print("Accuracy модели на тестовой выборке:", round(accuracy_forest_test,5), "Разница с показателем на валидационной выборке:", abs(round(accuracy_forest_test-best_accuracy_forest,5)))
print("Precision модели на тестовой выборке:", round(precision_forest_test,5), "Разница с показателем на валидационной выборке:", abs(round(precision_forest_test-precision_forest,5)))

Accuracy модели на тестовой выборке: 0.77605 Разница с показателем на валидационной выборке: 0.01866
Precision модели на тестовой выборке: 0.75926 Разница с показателем на валидационной выборке: 0.00107


_Проверим также модель `Решающее дерево`_

In [20]:
model_tree = joblib.load('model_tree.joblib')
prediction_tree_test = model_tree.predict(features_test)
accuracy_tree_test = accuracy_score(target_test, prediction_tree_test)
precision_tree_test = precision_score(target_test, prediction_tree_test, average='binary')

In [21]:
print("Accuracy модели на тестовой выборке:", round(accuracy_tree_test,5), "Разница с показателем на валидационной выборке:", abs(round(accuracy_tree_test-best_accuracy_tree,5)))
print("Precision модели на тестовой выборке:", round(precision_tree_test,5), "Разница с показателем на валидационной выборке:", abs(round(precision_tree_test-precision_tree,5)))

Accuracy модели на тестовой выборке: 0.75894 Разница с показателем на валидационной выборке: 0.02955
Precision модели на тестовой выборке: 0.74725 Разница с показателем на валидационной выборке: 0.06842


_И наконец проверим модель `Логистическая регрессия`_

In [22]:
prediction_logistic_test = model_logistic.predict(features_test)
accuracy_logistic_test = accuracy_score(target_test, prediction_logistic_test)
precision_logistic_test = precision_score(target_test, prediction_logistic_test, average='binary')

In [23]:
print("Accuracy модели на тестовой выборке:", round(accuracy_logistic_test,5), "Разница с показателем на валидационной выборке:", abs(round(accuracy_logistic_test-accuracy_logistic,5)))
print("Precision модели на тестовой выборке:", round(precision_logistic_test,5), "Разница с показателем на валидационной выборке:", abs(round(precision_logistic_test-precision_logistic,5)))

Accuracy модели на тестовой выборке: 0.69673 Разница с показателем на валидационной выборке: 0.00622
Precision модели на тестовой выборке: 0.69231 Разница с показателем на валидационной выборке: 0.0


_Дополним Таблицу №1 выше и создадим Таблицу №2 со сравнением результатов моделй на тестовой выборке_

In [24]:
model_comparision2 = pd.DataFrame({"model": ["Решающее дерево", "Случайный лес", "Логистическая"], 
                                   "accuracy_test": [accuracy_tree_test, accuracy_forest_test, accuracy_logistic_test],
                                   "precision_test": [precision_tree_test, precision_forest_test, precision_logistic_test]
                                 })

In [25]:
print("_ _ _ _ _ Табл.№1.Сравнение моделей (до)_ _ _ _ _ _ ")
print("_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ")
print(model_comparision)
print()
print("_ _ _ _Табл.№2.Сравнение моделей (после) _ _ _ _")
print("_ __ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _")
print(model_comparision2)

_ _ _ _ _ Табл.№1.Сравнение моделей (до)_ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
             model  accuracy_valid  precision_valid
0  Решающее дерево        0.788491         0.678832
1    Случайный лес        0.794712         0.760331
2    Логистическая        0.702955         0.692308

_ _ _ _Табл.№2.Сравнение моделей (после) _ _ _ _
_ __ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
             model  accuracy_test  precision_test
0  Решающее дерево       0.758942        0.747253
1    Случайный лес       0.776050        0.759259
2    Логистическая       0.696734        0.692308


In [26]:
## Приме.Код от ревьюера
model_forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=123456,
                       verbose=0, warm_start=False)

#### Вывод:
Мы видим, что `accuracy` и `precision` нашей модели больше целевого значения 0.75, а разница с этими же показателями на валидационной выборке не существенна. __Мы хорошо обучили модель, существенное переобучение/недообучение не наблюдается. :)__

<a id='step5'></a>

## 5. (бонус) Проверьте модели на адекватность

_Для проверки модели на адекватность оценим насколько слуйно созданная выборка нулей и единиц `dummy` способна предсказывать тариф_

In [27]:
accuracy_dummy = accuracy_score(target_test, target_dummy)
precision_dummy = precision_score(target_test, target_dummy, average='binary')

In [28]:
print("Accuracy dummy модели:", round(accuracy_dummy,5), "Разница с показателем модели случайный лес на тестовой выборке:", abs(round(accuracy_forest_test-accuracy_dummy,5)))
print("Precision dummy модели:", round(precision_dummy,5), "Разница с показателем модели случайный лес на тестовой выборке:", abs(round(precision_forest_test-precision_dummy,5)))

Accuracy dummy модели: 0.47123 Разница с показателем модели случайный лес на тестовой выборке: 0.30482
Precision dummy модели: 0.28528 Разница с показателем модели случайный лес на тестовой выборке: 0.47398


<a id='step5_last'></a>

_Используем библиотеку DummyClassifier_

In [29]:
model_dummy2 = DummyClassifier(strategy="most_frequent")
model_dummy2.fit(features_train, target_train)
prediction_dummy2 = model_logistic.predict(features_test)
accuracy_dummy2 = accuracy_score(target_test, prediction_dummy2)
precision_dummy2 = precision_score(target_test, prediction_dummy2, average='binary')

In [30]:
print("Accuracy dummy2 модели:", round(accuracy_dummy2, 5))
print("Precision dummy2 модели:", round(precision_dummy2, 5))

Accuracy dummy2 модели: 0.69673
Precision dummy2 модели: 0.69231


#### Вывод:
`Dummy модель` не обладает хоть какой нибудь предсказательной силой, поскольку правильно рекомендует тариф только в 49% случаев и попадает в 31% - это эффективность подбрасывания монетки и алгоритм никакой писать не надо.
___
Конечно же наша модель `Случайный лес` гораздо лучше справляется с рекомендацией тарифа. Правильность (`accuracy`) выше рандомного на 27.6%, а точность (`precision`) выше на 44%. __Еще раз убеждаемся, что модель составлена правильно.__

<a id='step6'></a>

## 6. Общий вывод:
1. Мы сравнили 3 модели машинного обучения и нашли наилучшую в данной задаче по показателям `accuracy` и `precision` - это модель `Случайный лес` c глубиной дерева 5 и количеством эстиматоров 30.
1. Модель справляется с рекомендацией тарифа лучше чем 2 другие: Дерево решения и Логистическая регрессия и что не маловажно гораздо лучше, чем простое подкидывание монетки.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
